In [261]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
file_train = "~/ds/proj3/tanzania-water-wells/data/raw/training-set-values.csv" #x_train, training set values data
file_target_train = "~/ds/proj3/tanzania-water-wells/data/raw/training-labels.csv" #y_train, training set labels
file_test = "~/ds/proj3/tanzania-water-wells/data/raw/test-set.csv" #x_test, test set data
features = pd.read_csv(file_train)
targets = pd.read_csv(file_target_train)
X_test = pd.read_csv(file_test)

In [180]:
from src.utilities import functions_used as func

In [263]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, random_state=42)

In [265]:
joined_df = X_train.join(y_train, rsuffix='_r')

In [266]:
joined_test = X_test.join(y_test, rsuffix='_r')

In [247]:
con_year_nonz = joined_test.copy()
con_year_nonz['construction_year'] = con_year_nonz['construction_year'].replace(0, np.nan)

In [248]:
avg_con_years = pd.DataFrame(con_year_nonz.groupby(['extraction_type']).mean()['construction_year'])
features_df = joined_test.copy()
features_df = features_df.join(avg_con_years, rsuffix= '_avg', on = 'extraction_type')
features_df['construction_year'] = features_df.apply(con_year, axis = 1)

In [249]:
features_df['con_year_mod'] = 0
features_df['con_year_mod'] = features_df.apply(mark_con_year_avg, axis = 1)

In [250]:
model_df = features_df[features_list]

In [251]:
long_drop_df = drop_zero_long(model_df)

In [252]:
long_drop_df = long_drop_df.drop(long_drop_df.loc[long_drop_df['construction_year'].isna() == True].index)

In [253]:
target_df = long_drop_df['status_group']
feature_df = long_drop_df.drop(['status_group'], axis = 1)

In [254]:
obj_list = func.obj_lister(feature_df)
feature_df = feature_df.drop(obj_list, axis = 1)

In [255]:
test_feature_df = feature_df.copy()
test_target_df = target_df.copy()

In [245]:
## DONT RERUN
train_feature_df = feature_df.copy()
train_target_df = target_df.copy()

### Trying to just model based on this new feature engineering

In [256]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50, random_state=42, bootstrap=True, max_depth = 50)

In [257]:
rfc.fit(train_feature_df, train_target_df)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [172]:
obj_list = func.obj_lister(feature_df)
obj_df = model_df[obj_list]
obj_df = NaN_cleaning(obj_df)
ohe = OneHotEncoder()
ohe_obj = ohe.fit_transform(obj_df).toarray()
obj_df = pd.DataFrame(ohe_obj)

Cleaning the data
Check: Number of rows with nulls: 0


In [258]:
rfc_predicts = rfc.predict(test_feature_df)

In [259]:
rfc.score(test_feature_df,test_target_df)

0.7190651735410726

In [262]:
confusion_matrix(test_target_df,rfc_predicts)

array([[6270,  260, 1332],
       [ 478,  276,  233],
       [1602,  134, 3792]])

Without above engineering to compare

In [272]:
# This is our list of features, plus the target (required for our preprocessing step)

alt_features_list = ['gps_height', 'longitude', 'latitude', 
       'region_code', 'district_code', 'population', 'status_group']   


In [267]:
long_drop_train = drop_zero_long(joined_df)

In [268]:
long_drop_test = drop_zero_long(joined_test)

In [273]:
long_drop_test = long_drop_test[alt_features_list]

In [274]:
long_drop_train = long_drop_train[alt_features_list]

In [276]:
X_train = long_drop_train.drop(['status_group'], axis = 1)

In [277]:
y_train = long_drop_train['status_group']

In [278]:
X_test = long_drop_test.drop(['status_group'], axis = 1)
y_test = long_drop_test['status_group']

In [279]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50, random_state=42, bootstrap=True, max_depth = 50)

In [281]:
rfc.fit(X_train, y_train)
rfc_predicts = rfc.predict(X_test)
rfc.score(X_test,y_test)

0.6978507338109481

In [282]:
confusion_matrix(y_test,rfc_predicts)

array([[6238,  217, 1407],
       [ 507,  268,  212],
       [1857,  144, 3527]])

# Meh, it's still breaking

In [ ]:
long_drop_df = drop_zero_long(model_df)

_____

In [17]:
avg_con_years = pd.DataFrame(con_year_nonz.groupby(['extraction_type']).mean()['construction_year'])

In [24]:
features_df = features.copy()
features_df = features_df.join(avg_con_years, rsuffix= '_avg', on = 'extraction_type')

In [139]:
def con_year(row: pd.DataFrame) -> int:
        ## This function replaces NaN values (earlier converted to zeros) of construction_year
        ## to the average of the construction year (construction_year_avg) for a given record's extraction type 
        if (row['construction_year'] == 0):
            return row['construction_year_avg']
        else:
            return row['construction_year']

In [140]:
def mark_con_year_avg(row):
    if (row['construction_year'] == row['construction_year_avg']):
        return 1
    else:
        return 0

In [36]:
features_df['construction_year'] = features_df.apply(con_year, axis = 1)

In [40]:
features_df['con_year_mod'] = 0
features_df['con_year_mod'] = features_df.apply(mark_con_year_avg, axis = 1)

In [44]:
features_df['con_year_mod'] = features_df.apply(mark_con_year_avg, axis = 1)

In [49]:
features_df['con_year_mod'].value_counts()

0    38681
1    20719
Name: con_year_mod, dtype: int64

In [117]:
# haven't actually done this part
pop_nonz = features.copy()
pop_nonz['population'] = pop_nonz['population'].replace(0, np.nan)
pop_nonz['population'] = pop_nonz['population'].replace(1, np.nan)
pop_nonz['avg_pop'] = con_year_nonz['population'].mean()



In [74]:
def avg_pop(row: pd.DataFrame) -> int:
        ## This function replaces NaN values (earlier converted to zeros) of construction_year
        ## to the average of the construction year (construction_year_avg) for a given record's extraction type 
        if (row['population'] == 0) or (row['population'] == 1):
            return row['avg_pop']
        else:
            return row['population']
        
def mark_con_year_avg(row):
    if (row['population'] == row['avg_pop']):
        return 1
    else:
        return 0

In [167]:
# This is our list of features, plus the target (required for our preprocessing step)

features_list = ['basin', 'region', 'scheme_management', 'scheme_name',
       'extraction_type', 'management', 'payment', 'water_quality', 'quantity',
       'source', 'waterpoint_type','gps_height', 'longitude', 'latitude', 
       'region_code', 'district_code', 'population', 'construction_year', 'con_year_mod', 'status_group']   


In [56]:
model_df = features_df[features_list]

In [57]:
obj_list = func.obj_lister(model_df)

In [76]:
def obj_preprocessing(df, obj_list, ohe, train = True):
    '''
    '''
    df_current = df[obj_list]
    # Clean the df if there are NaNs
    df = NaN_cleaning(df_current)
    #OHE data
    array_current = ohe_data(df, ohe, train)
    #return a dataframe of the OHE data
    return pd.DataFrame(array_current)

def ohe_data(df, ohe, train):
    #OHE the data
    print('One hot encoding data')
    if train:
        array_current = ohe.fit_transform(df).toarray()
    else:
        array_current = ohe.transform(df).toarray()
    return array_current

def NaN_cleaning(df):
    # Replace NaN with "unknown" bin
    print('Cleaning the data')
    df = df.replace(np.nan, 'unknown')
    print(f'Check: Number of rows with nulls: {len(df[df.isna().any(axis=1)])}')
    return df

In [89]:
obj_df = model_df[obj_list]
obj_df = NaN_cleaning(obj_df)
ohe = OneHotEncoder()
ohe_obj = ohe.fit_transform(obj_df).toarray()
obj_df = pd.DataFrame(ohe_obj)

In [90]:
obj_df = NaN_cleaning(obj_df)

Cleaning the data
Check: Number of rows with nulls: 0


In [91]:
ohe = OneHotEncoder()

In [92]:
ohe_obj = ohe.fit_transform(obj_df).toarray()

In [93]:
obj_df = pd.DataFrame(ohe_obj)

In [94]:
obj_df

,0,1,2,3,4,5,6,7,8,9,...,2797,2798,2799,2800,2801,2802,2803,2804,2805,2806
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59396,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59397,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59398,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [99]:
model_df = model_df.drop(obj_list, axis = 1)
model_df = model_df.join(obj_df)

In [101]:
model_df

,gps_height,longitude,latitude,region_code,district_code,population,construction_year,con_year_mod,0,1,...,2797,2798,2799,2800,2801,2802,2803,2804,2805,2806
0,1390,34.938093,-9.856322,11,5,109,1999.000000,0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1399,34.698766,-2.147466,20,2,280,2010.000000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,686,37.460664,-3.821329,21,4,250,2009.000000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,263,38.486161,-11.155298,90,63,58,1986.000000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,31.130847,-1.825359,18,1,0,1995.965949,1,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,1210,37.169807,-3.253847,3,5,125,1999.000000,0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59396,1212,35.249991,-9.070629,11,4,56,1996.000000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59397,0,34.017087,-8.750434,12,7,0,1996.671370,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59398,0,35.861315,-6.378573,1,4,0,1999.421166,1,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [103]:
joined_df = model_df.join(targets)

In [106]:
joined_df = joined_df.drop(['id'], axis = 1)

In [107]:
joined_df

,gps_height,longitude,latitude,region_code,district_code,population,construction_year,con_year_mod,0,1,...,2798,2799,2800,2801,2802,2803,2804,2805,2806,status_group
0,1390,34.938093,-9.856322,11,5,109,1999.000000,0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional
1,1399,34.698766,-2.147466,20,2,280,2010.000000,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional
2,686,37.460664,-3.821329,21,4,250,2009.000000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,functional
3,263,38.486161,-11.155298,90,63,58,1986.000000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,non functional
4,0,31.130847,-1.825359,18,1,0,1995.965949,1,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,1210,37.169807,-3.253847,3,5,125,1999.000000,0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional
59396,1212,35.249991,-9.070629,11,4,56,1996.000000,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional
59397,0,34.017087,-8.750434,12,7,0,1996.671370,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,functional
59398,0,35.861315,-6.378573,1,4,0,1999.421166,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,functional


In [108]:
def drop_zero_long(df):
    return df.drop(df[df.longitude==0].index)

In [109]:
long_drop_df = drop_zero_long(joined_df)

In [111]:
targets = long_drop_df['status_group']

In [114]:
features = long_drop_df.drop(['status_group'], axis = 1) 

In [115]:
features.shape

(57588, 2815)

In [116]:
targets.shape

(57588,)